In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
# Afficher le répertoire courant
print("Répertoire courant :", os.getcwd())

Répertoire courant : C:\Users\flero\Downloads


In [8]:
# Changer de répertoire (remplacez le chemin par celui de votre dossier)
os.chdir(r'D:\HEC\Msc BI\Data Mining\Devoir Hiver')
print("Nouveau répertoire :", os.getcwd())

Nouveau répertoire : D:\HEC\Msc BI\Data Mining\Devoir Hiver


In [10]:

# Importation des données
# Ici, nous utilisons pd.read_csv() pour lire les fichiers texte.
# L'argument sep=r'\s+' précise que les colonnes sont séparées par des espaces.
mobiletrain = pd.read_csv("train.csv", sep=',')
mobiletest  = pd.read_csv("test.csv", sep=',')

# Affichage des dimensions du DataFrame mobiletrain
# L'attribut .shape renvoie un tuple (nombre de lignes, nombre de colonnes)
print("Dimensions de mobiletrain :", mobiletrain.shape)


Dimensions de mobiletrain : (5000, 17)


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [16]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsOneClassifier


In [18]:
# Nettoyage des colonnes
mobiletrain.columns = mobiletrain.columns.str.strip()
mobiletest.columns = mobiletest.columns.str.strip()

# Création de variables dérivées (feature engineering simple)
mobiletrain['avg_score_per_session'] = mobiletrain['totscore'] / (mobiletrain['numsessions'] + 1)
mobiletest['avg_score_per_session'] = mobiletest['totscore'] / (mobiletest['numsessions'] + 1)

# Séparer les features et la cible
X = pd.get_dummies(mobiletrain.drop(columns=['y']))
y = mobiletrain['y']

# Encodage de la cible
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Adapter X_test aux colonnes d'entraînement
X_test = pd.get_dummies(mobiletest.drop(columns=['id']))
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Définition du modèle XGBoost (modèle de base pour chaque paire OvO)
base_model = xgb.XGBClassifier(
    objective='binary:logistic',  # binaire pour chaque sous-modèle OvO
    eval_metric='logloss',
    n_estimators=688,
    learning_rate=0.19506627095212276,
    max_depth=4, 
    min_child_weight=10, 
    subsample=0.6602379386300246,
    colsample_bytree=0.9567880663184878,
    gamma=0.8519961776491365,
    use_label_encoder=False,
    random_state=42
)

# Envelopper dans un OneVsOneClassifier
ovo_model = OneVsOneClassifier(base_model)

# Entraînement du modèle OvO
ovo_model.fit(X, y_encoded)

# Prédictions sur le jeu de test
y_test_pred_encoded = ovo_model.predict(X_test)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)

# Création du fichier de soumission
submission_xgb = pd.DataFrame({
    'id': mobiletest['id'],
    'Prediction': y_test_pred
})

# Export du fichier CSV
submission_xgb.to_csv("submission_xgboost_ovo.csv", index=False)

print("✅ Fichier 'submission_xgboos_ovot.csv' généré avec XGBoost pour Kaggle.")


C:\Users\flero\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:21:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\flero\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:21:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\flero\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:21:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\flero\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:21:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

✅ Fichier 'submission_xgboos_ovot.csv' généré avec XGBoost pour Kaggle.
